In [21]:
f = open("bloom-syndrome-protein.txt", "r")
lines = f.readlines()
genome = ''.join(''.join(lines).split())
genome

'gaagatgctcaggaaagtgactctctgaaaactcatttggaagatgaaagagataatagcgaaaagaagaagaatttggaagaagctgaattacattcaactgagaaagttccatgtattgaatttgatgatgatgattatgatacggattttgttccaccttctccagaagaaattatttctgcttcttcttcctcttcaaaatgccttagtacgttaaaggaccttgacacctctgacagaaaagaggatgttcttagcacatcaaaagatcttttgtcaaaacctgagaaaatgagtatgcaggagctgaatccagaaaccagcacagactgtgacgctagacagataagtttacagcagcagcttattcatgtgatggagcacatctgtaaattaattgatactattcctgatgataaactgaaacctttggattgtgggaacgaactgcttcagcagcggaacataagaaggaaacttctaacggaagtagattttaataaaagtgatgccagtcttcttggctcattgtggagatacaggcctgattcacttgatggccctatggagggtgattcctgccctacagggaattctatgaaggagttaaatttttcacaccttccctcaaattctgtttctcctggggactgtttactgactaccaccctaggaaagacaggattctctgccaccaggaagaatctttttgaaaggcctttattcaatacccatttacagaagtcctttgtaagtagcaactgggctgaaacaccaagactaggaaaaaaaaatgaaagctcttatttcccaggaaatgttctcacaagcactgctgtgaaagatcagaataaacatactgcttcaataaatgacttagaaagagaaacccaaccttcctatgatattgataattttgacatagatgactttgatgatgatgatgactgggaagacataatgcataatttagcagccagcaaatcttccacagct

In [17]:
MAX_RUN = 4
genome_size = len(genome)
compressed = ""
i = 0
while i < genome_size: 
    run_length = 1
    k = i+1
    while k < genome_size:
        if genome[i]==genome[k]:
            run_length += 1
            k += 1
        else:
            break
   
    if run_length == 1:
        compressed += genome[i]
    else:
        compressed += str(run_length) + genome[i]
    i = k   
len(compressed)

3139

In [22]:
compressed

'g2agatgctca2g3agtgactctctg4actca3t2g2agatg3agagat2atagcg4ag2ag2ag2a3t2g2ag2agctg2a2taca2tc2actgag3ag2t2catgta2tg2a3tgatgatgatga2tatgatac2ga4tg2t2ca2c2tct2cag2ag3a2ta3tctgc2tc2tc2t2ctc2tc4atg2c2tagtacg2t3a2ga2c2tgaca2ctctgacag4aga2gatg2tc2tagcacatc4agatc4tgtc4a2ctgag4atgagtatgca2gagctg2at2cag3a2cagcacagactgtgacgctagacagat2ag3tacagcagcagc2ta2tcatgtgat2gagcacatctgt3a2t2a2tgatacta2t2ctgatgat3actg3a2c3t2ga2tgt3g2acg2actgc2tcagcagc2g2acat2ag2a2g3ac2tct2ac2g2agtaga4t2at4agtgatg2cagtc2tc2t2gctca2tgt2gagataca2g2ctga2tcac2tgat2g3ctat2ga3gtga2t2ctg3ctaca3g2a2tctatg2a2gag2t3a5tcaca2c2t3ctc3a2tctg3tct2ct4gactg3tactgacta2ca3cta2g3agaca2ga2tctctg2ca2ca2g2ag2atc5tg3a2g2c3ta2tc2ata3ca3tacag2agt2c3tgt2agtagc2act3gctg3aca2c2agacta2g9atg3agctc2ta3t3ca2g3atg2tctcac2agcactgctgtg3agatcag2at3acatactgc2tc2at3atgac2tag3agag3a3c2a2c2t2ctatgata2tgat2a4tgacatagatgac3tgatgatgatgatgact3g2agacat2atgcat2a3tagcag2cagc3atc2t2cacagctg2ctatc2a3catc2a2g2a2gtc2g2c2a2t3atcagtatcag3agac3t2ctcag2c2agacagactgtc2t2cagtgtcatctac